# LSTM

In [1]:
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn as nn

import numpy as np
import pandas as pd
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

from torch.utils.tensorboard import SummaryWriter

from scipy.stats import mode

from modules.pytorchtools import EarlyStopping 

import os
import shutil
import glob
import random

main_folder = "../data"

folder = "./runs/LSTM_1"

try:
    shutil.rmtree(folder, ignore_errors=True)
except:
    pass

try:
    os.mkdir(folder)
except:
    pass

writer = SummaryWriter(folder)

In [2]:
LABELS = [
    "No_action",
    "Prendi",
    "Rilascia",
    "Premi"
]

In [3]:
class2idx = {
    "No_action": 0,
    "Prendi": 1,
    "Rilascia": 2,
    "Premi": 3
}

idx2class = {v: k for k, v in class2idx.items()}

In [4]:
def create_set(folder_set):
    csv:list = []
    for file in glob.glob(folder_set + "/*.csv"):
        csv.append(file)
    data = []
    target = []
    for fcsv in csv:
        data_video = pd.read_csv(fcsv, usecols = [i for i in range(156)]).to_numpy()
        target_video = pd.read_csv(fcsv, usecols = ["TARGET"])
        target_video["TARGET"].replace(class2idx, inplace=True)
        data.append(data_video)
        target.append(target_video.to_numpy())
    return (np.array(data, dtype=object), np.array(target, dtype=object))

In [5]:
folder_set = [[main_folder + "/train_set", main_folder + "/train.csv"], [main_folder + "/test_set", main_folder + "/test.csv"], [main_folder + "/val_set", main_folder + "/val.csv"]]

train_array, train_label_array = create_set(folder_set[0][0])
test_array, test_label_array = create_set(folder_set[1][0])
val_array, val_label_array = create_set(folder_set[2][0])

In [6]:
class ClassifierDataset(Dataset):
    
    def __init__(self, array, label, index_data, window):
        self.index_data = index_data
        self.array = array
        self.label = label
        self.window = window
        
    def __getitem__(self, index):
        file_index = self.index_data[index][0].tolist()
        i = self.index_data[index][1]
        j = self.index_data[index][2]
        if i == j:
            x_data = [self.array[file_index][i]] * self.window
            y_data = self.label[file_index][i]
            X = torch.from_numpy(np.array(x_data).astype(float)).float()
            Y = torch.from_numpy(np.array([y_data]).astype(int)).long()
            return X, Y
        elif j-i < self.window-1:
            x_data = [self.array[file_index][i]] * (self.window-j)
            x_data.extend(self.array[file_index][(i+1):(j+1)])
            x_data = np.array(x_data).astype(float)
        else:
            x_data = self.array[file_index][i:(j+1)]
            k = j-1
            while len(x_data) > self.window:
                x_data = self.array[file_index][i:(k+1)]
                k -= 1
        X = torch.from_numpy(x_data).float()
        y_data = self.label[file_index][i:(j+1)]
        y_mode = mode(y_data)[0][0]
        Y = torch.from_numpy(np.array([y_mode])).long()
        return X, Y
        
    def __len__ (self):
        return len(self.index_data)

In [7]:
def create_dataset(array_data, label_data, window):
    X = []
    files_num = len(array_data)
    for index, array, label in zip(range(files_num), array_data, label_data):
        l = array.shape[0]
        i = 0
        while l - i >= window:
            j = i + window
            index_window = np.array([index, i, j]).astype(int)
            X.append(index_window)
            i += window
        if i < l:
            w = l - i
            i = i - w - window
            j = i + window
            index_window = np.array([index, i, j]).astype(int)
            X.append(index_window)
    X_data = torch.from_numpy(np.array(X).astype(int))
    return ClassifierDataset(array_data, label_data, X_data, window)

In [8]:
def create_dataset2(array_data, label_data, window):
    X = []
    files_num = len(array_data)
    for index, array, label in zip(range(files_num), array_data, label_data):
        l = array.shape[0]
        i = -1
        w = [0] * window
        while i < l:
            i += 1 
            w.pop(0)
            w.append(i)
            index_window = np.array([index, w[0], i]).astype(int)
            X.append(index_window)
    X_data = torch.from_numpy(np.array(X).astype(int))
    return ClassifierDataset(array_data, label_data, X_data, window)

In [9]:
window_list = [(i) for i in range(1, 31)]

In [10]:
random.shuffle(window_list)

In [11]:
batch_size_list = [4, 8, 16, 32, 64, 128, 256, 512]

In [12]:
learning_rate_list = [(random.uniform(0.01, 0.0000001)) for _ in range(0, 60)]

In [13]:
layer_list = [1, 2, 3, 4]

In [14]:
hidden_list = [(i) for i in range(4, 513)]
random.shuffle(hidden_list)

In [15]:
index_name = 16

In [16]:
EPOCHS = 200
window = 23
BATCH_SIZE = 512
LEARNING_RATE = 0.04
NUM_LAYER = 2
NUM_HIDDEN = 16
NUM_FEATURES = 156
NUM_CLASSES = 4
model_name = ""

In [17]:
class LSTM(nn.Module):
    def __init__(self, input_size, window, output_size, hidden_layer_size, num_layers):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_layer_size
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers)
        self.regressor = nn.Linear(hidden_layer_size, output_size)
        #self.bn = nn.BatchNorm1d(window)
        
    def forward(self, x, hidden=None):
        if hidden is not None:
            h0 = hidden[0]
            c0 = hidden [1]
        else:
            h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(device)
        e = x.view(x.size(1), x.size(0), x.size(2))
        h, hn = self.lstm(e, (h0, c0))
        h = h.view(h.size(1), h.size(0), h.size(2))
        h = h[:,-1,:]
        h = self.regressor(h)
        return h, hn

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

In [20]:
def accuracy(y_pred, y_true):
    return accuracy_score(y_true, y_pred) * 100

In [21]:
def f1_s(y_pred, y_true):
    return f1_score(y_true, y_pred, average=None, zero_division=1, labels=[0,1,2,3])

In [ ]:
while True:
    window = random.choice(window_list)
    BATCH_SIZE = random.choice(batch_size_list)
    LEARNING_RATE = random.choice(learning_rate_list)
    NUM_LAYER = random.choice(layer_list)
    NUM_HIDDEN = random.choice(hidden_list)
    model_name = str(index_name) + "mymodel.pt"
    print("\n\nREPORT:")
    print("WINDOW:", window)
    print("BATCH_SIZE:", BATCH_SIZE)
    print("LEARNING_RATE:", LEARNING_RATE)
    print("NUM_LAYER:", NUM_LAYER)
    print("NUM_HIDDEN:", NUM_HIDDEN)
    print("MODEL_NAME:", model_name)
    try:
        train_dataset = create_dataset(train_array, train_label_array, window)
        test_dataset = create_dataset2(test_array, test_label_array, window)
        val_dataset = create_dataset2(val_array, val_label_array, window)
        train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
        val_loader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
        model = LSTM(input_size=NUM_FEATURES, window=window, output_size=NUM_CLASSES, hidden_layer_size=NUM_HIDDEN, num_layers=NUM_LAYER)
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
        n_data_size_test = test_dataset.index_data.size()[0]
        n_data_size_train = train_dataset.index_data.size()[0]
        n_data_size_val = val_dataset.index_data.size()[0]
        accuracy_stats = {
            'train': [],
            "val": []
        }

        loss_stats = {
            'train': [],
            "val": []
        }

        f1_score_No_action = {
            'train': [],
            "val": []
        }

        f1_score_Prendi = {
            'train': [],
            "val": []
        }

        f1_score_Rilascia = {
            'train': [],
            "val": []
        }

        f1_score_Premi = {
            'train': [],
            "val": []
        }
        print("Begin training: ", str(index_name))
        for e in tqdm(range(1, EPOCHS+1)):

            # TRAINING
            train_epoch_loss = 0
            train_epoch_acc = 0
            train_epoch_f1_No_action = 0
            train_epoch_f1_Prendi = 0
            train_epoch_f1_Rilascia = 0
            train_epoch_f1_Premi = 0
            train_pred_lab = []
            train_true_lab = []

            model.train()

            hidden_train = None

            for X_train_batch, y_train_batch in train_loader:
                X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
                y_train_batch = torch.squeeze(y_train_batch)
                optimizer.zero_grad()

                y_train_pred, _ = model(X_train_batch, hidden_train)

                #h0, c0 = hidden_train
                #h0.detach_(), c0.detach_()
                #hidden_train = (h0, c0)

                train_loss = criterion(y_train_pred, y_train_batch)
                train_pred_lab.append(y_train_pred)
                train_true_lab.append(y_train_batch)

                train_loss.backward()
                optimizer.step()

                train_epoch_loss += train_loss.item()

            y_pred_tags = []
            y_true_tags = []
            for (p, t) in zip(train_pred_lab, train_true_lab):
                _, y_tags = torch.max(p, dim = 1)
                y_pred_tags.extend(y_tags.tolist())
                y_true_tags.extend(t.tolist())

            train_acc = accuracy(y_pred_tags, y_true_tags)
            train_f1 = f1_s(y_pred_tags, y_true_tags)

            train_epoch_acc += train_acc.item()
            train_epoch_f1_No_action += train_f1[0].item()
            train_epoch_f1_Prendi += train_f1[1].item()
            train_epoch_f1_Rilascia += train_f1[2].item()
            train_epoch_f1_Premi += train_f1[3].item()
        print("Done: ", str(index_name))
        y_pred_list = []
        y_true_list = []

        with torch.no_grad():
            model.eval()
            for X_batch, y_batch in test_loader:
                X_batch = X_batch.to(device)
                y_test_pred, _ = model(X_batch)
                guess, guess_i = categoryFromOutput(y_test_pred)
                y_pred_list.append(guess_i)
                y_true_list.append(y_batch[0][0].tolist()[0])
        accuracy_test = accuracy(y_pred_list, y_true_list)
        f1_score_list = f1_s(y_pred_list, y_true_list)
        print("Accuracy: ", accuracy_test)
        print("F1 - No_action: ", f1_score_list[0])
        print("F1 - Prendi: ", f1_score_list[1])
        print("F1 - Rilascia: ", f1_score_list[2])
        print("F1 - Premi: ", f1_score_list[3])
        #if accuracy_test >= 65:
        torch.save(model.state_dict(), main_folder + "/random_model/" + model_name)
        index_name += 1
    except:
        print("Error.")
        continue



REPORT:
WINDOW: 12
BATCH_SIZE: 16
LEARNING_RATE: 0.007808660957658549
NUM_LAYER: 1
NUM_HIDDEN: 395
MODEL_NAME: 16mymodel.pt
Begin training:  16


  0%|          | 0/200 [00:00<?, ?it/s]

Done:  16
Accuracy:  60.7413647851727
F1 - No_action:  0.8784621744522529
F1 - Prendi:  0.4018126888217522
F1 - Rilascia:  0.40249249942303256
F1 - Premi:  0.45399254463409844


REPORT:
WINDOW: 11
BATCH_SIZE: 128
LEARNING_RATE: 0.008660302628860384
NUM_LAYER: 3
NUM_HIDDEN: 391
MODEL_NAME: 17mymodel.pt
Begin training:  17


  0%|          | 0/200 [00:00<?, ?it/s]

Done:  17
Accuracy:  53.091828138163436
F1 - No_action:  0.8352206169465053
F1 - Prendi:  0.44952803997779006
F1 - Rilascia:  0.0
F1 - Premi:  0.0


REPORT:
WINDOW: 7
BATCH_SIZE: 16
LEARNING_RATE: 0.007234481004054133
NUM_LAYER: 3
NUM_HIDDEN: 90
MODEL_NAME: 18mymodel.pt
Begin training:  18


...

  0%|          | 0/200 [00:00<?, ?it/s]

Error.


REPORT:
WINDOW: 24
BATCH_SIZE: 64
LEARNING_RATE: 0.004829401118036806
NUM_LAYER: 3
NUM_HIDDEN: 321
MODEL_NAME: 46mymodel.pt
Begin training:  46


  0%|          | 0/200 [00:00<?, ?it/s]

Done:  46
Accuracy:  50.488626790227464
F1 - No_action:  0.756692037971436
F1 - Prendi:  0.4131665569453588
F1 - Rilascia:  0.0
F1 - Premi:  0.0


REPORT:
WINDOW: 13
BATCH_SIZE: 128
LEARNING_RATE: 0.0021384036546737324
NUM_LAYER: 2
NUM_HIDDEN: 154
MODEL_NAME: 47mymodel.pt
Begin training:  47


  0%|          | 0/200 [00:00<?, ?it/s]